# Data Exploration - Continued

In this notebook, we will explore our data beyond what we saw in our data collcetion preview.

## Loading our data

In our data collection, we pickled each dataframe. We'll load these back in:

In [12]:
import pandas as pd
import praw as praw
from praw.models import MoreComments
import requests as requests
import pandas as pd

In [4]:
#askReddit
askReddit_df = pd.read_pickle("./data/askReddit_df.pkl")
askReddit_hot_df = pd.read_pickle("./data/askReddit_hot_df.pkl")

#politics
politics_df = pd.read_pickle("./data/politics_df.pkl")
politics_hot_df = pd.read_pickle("./data/politics_hot_df.pkl")
#soccer

soccer_df = pd.read_pickle("./data/soccer_df.pkl")
soccer_hot_df = pd.read_pickle("./data/soccer_hot_df.pkl")

#uplifting news

uplifting_df = pd.read_pickle("./data/uplifting_df.pkl")
uplifting_hot_df = pd.read_pickle("./data/uplifting_hot_df.pkl")

#music

music_df = pd.read_pickle("./data/music_df.pkl")
music_hot_df = pd.read_pickle("./data/music_hot_df.pkl")

Let's verify that the data was loaded in correctly:

In [5]:
askReddit_df.head(10)

,author,title,text,numComments,comments
id,,,,,
122xgeb,kamal31121984,Anyone who works 40 hours a week for minimum w...,,25,"(jds8enf, jds84e4, jds7zjy, jds936a, jds813w, ..."
122yg9v,Routine_Camp7308,What do you consider a successful day for your...,,4,"(jdscos2, jdsdd8t, jdsd0bz, jdsel9r)"
122xxz4,BrudderO,What’s your favourite way to be seduced?,,9,"(jdsbg98, jdsa86a, jdsai9g, jdscykh, jdsaadi, ..."
122xti7,vicelordjohn,If you were sentenced to death and could choos...,,44,"(jds9qka, jds9qk3, jds9p0d, jds9pgt, jds9lma, ..."
122y4tz,MrFantastic74,How can North America get rid of tipping cultu...,,30,"(jdsb0k6, jdscykm, jdsb79k, jdsdq3w, jdsb099, ..."
122xerf,Impostata,"A-Arons of Reddit, how have you done messed up?",,3,"(jds7snq, jds7xb1, jds7xg0)"
122x0ev,FairtalkOfficial,What year do you believe AI will completely re...,,26,"(jds67us, jds6svt, jds9akd, jds5s28, jds77hg, ..."
122xk5t,Agitated-Pilot-1897,What’s the best place in your house for hiding...,,29,"(jds8lc2, jds8gx6, jds8eim, jds8w3o, jdsaqne, ..."
122yaam,puce_3000,What’s your favorite “grandmas remedy”?,,6,"(jdsbxdf, jdsbyqg, jdsc2vv, jdscf2h, jdscr51, ..."


Yep! 

#### Data Format

By now, we have our data format sorted. Each dataframe has 6 columns:

1. ID
2. Author
3. Title
4. Text
5. numComments
6. comments -  Each comment has its own 'comment tree'

Let's look at our comments again:

In [75]:
for comment in askReddit_hot_df.comments:
    comment = comment.replace_more(limit=0)
vars(askReddit_df.loc['122x0ev'].comments[0])

{'_replies': <praw.models.comment_forest.CommentForest at 0x1ad7b880640>,
 '_submission': Submission(id='122x0ev'),
 '_reddit': <praw.reddit.Reddit at 0x1ad7ba020d0>,
 'subreddit_id': 't5_2qh1i',
 'approved_at_utc': None,
 'author_is_blocked': False,
 'comment_type': None,
 'awarders': [],
 'mod_reason_by': None,
 'banned_by': None,
 'author_flair_type': 'text',
 'total_awards_received': 0,
 'subreddit': Subreddit(display_name='AskReddit'),
 'author_flair_template_id': None,
 'likes': None,
 'user_reports': [],
 'saved': False,
 'id': 'jds67us',
 'banned_at_utc': None,
 'mod_reason_title': None,
 'gilded': 0,
 'archived': False,
 'collapsed_reason_code': None,
 'no_follow': False,
 'author': Redditor(name='maler27'),
 'can_mod_post': False,
 'created_utc': 1679859851.0,
 'send_replies': True,
 'parent_id': 't3_122x0ev',
 'score': 1,
 'author_fullname': 't2_37mpif7x',
 'approved_by': None,
 'mod_note': None,
 'all_awardings': [],
 'collapsed': False,
 'body': 'Saw a Star Wars movie year

Here we can see all attributes that a comment object has. If we use the list() function as implemented, it displays comments breadth-first. This means that the first comment returned in our list is likely unrelated to the second, and so on. To minimize this issue, we can modify the function to print depth-first (for readability) and include how many times each comment was replied to:

In [93]:
def list_dfs(comm):  
        """Return a DFS list of all Comments.  
        This list may contain :class:`.MoreComments` instances if  
        :meth:`.replace_more` was not called first.  
        """  
        comment_counts = {}
        comments = []
        queue = list(comm)
        while queue:
            comment = queue.pop(0)
            comments.append(comment)
            comment_counts[comment.id] = 1
            if(comment.parent_id[3:] in comment_counts):
                comment_counts[comment.parent_id[3:]] += 1
            else:
                comment_counts[comment.parent_id[3:]] = 1
            if not isinstance(comment, MoreComments):
                queue[0:0] = comment.replies
        return comments

In [94]:
for comment in list_dfs(askReddit_df.loc['122x0ev'].comments):
    # Using ____ to demarcate different comments
    print(comment.id+ ' '+ comment.parent_id+ ' ' + comment.body + "\n _________")

jds67us t3_122x0ev Saw a Star Wars movie years ago that felt like I was watching a cartoon the entire time
 _________
jds7os1 t1_jds67us Looks like the AI takeover has already begun if they're making Jar Jar Binks look realistic.
 _________
jds6svt t3_122x0ev CGI hasn’t put the movie camera manufacturers out of a business. Nor has it made actors obsolete. 

AI is going to have a hard time replacing entire industries just like every other technological advancement has. It could change cinema substantially, but it will never kill the camera.
 _________
jds9akd t3_122x0ev I don't think it will completely replace it, but it will lower some costs, eliminate some jobs and reduce the time (and probably the quality) of movies in 10-20 years.
 _________
jds5s28 t3_122x0ev 2050
 _________
jds77hg t3_122x0ev Everyone here, do yourself a favor and watch the “Netflix Bots write movies” series on YouTube by Netflix is a Joke
 _________
jds814j t3_122x0ev There hasn’t been any type of intelligence, a

We can see a hint of conversation in these replies, as opposed to the BFS, which was a bit jumbled. Let's visualize this by looking exclusively at the first 2 entries: 

In [58]:
for comment in list_dfs(askReddit_df.loc['122x0ev'].comments)[:2]:
    # Using ____ to demarcate different comments
    print(comment.id+ ' '+ comment.parent_id+ ' ' + comment.body + "\n _________")

jds67us t3_122x0ev Saw a Star Wars movie years ago that felt like I was watching a cartoon the entire time
 _________
jds7os1 t1_jds67us Looks like the AI takeover has already begun if they're making Jar Jar Binks look realistic.
 _________


The id listed first is the comment's id, while the second is an identifier for the parent. The comment jds7os1 is replying to jds67us. This is a short chain, so let's look at one of the hotter posts:

In [74]:
print(askReddit_hot_df.iloc[1].title+"\n\n")
for comment in list_dfs(askReddit_hot_df.iloc[1].comments)[:10]:
    # Using ____ to demarcate different comments
    print(comment.id+ ' '+ comment.parent_id[3:]+ ' ' + comment.body + "\n _________")

What are some of the biggest scams to have happened in history?


jdraly2 122pfb9 
Ticketmaster.
 _________
jdrt7to jdraly2 My hate for Ticketmaster is beyond anything in the world.
 _________
jds17yc jdrt7to And they've got significantly worse in the post COVID era.
 _________
jds3k8k jdraly2 I can't even see a show for a reasonable price.  I call bullshit.  Ticketmaster has destroyed live music.
 _________
jds40cb jds3k8k What is reasonable price?  Seems like people are willing to pay thousands for a Taylor swift concert. So… reasonable price would be in the thousands.
 _________
jdscyla jds40cb I believe that tickets to see the National Symphony Orchestra at the Kennedy Center in Washington, D.C. run between $35 and $99.  That should be the reasonable price for every concert.
 _________
jdsd849 jdscyla You implying the national symphony orchestra is on the same level as Taylor Swift?
 _________
jdse0ex jdsd849 Yes
 _________
jdrwpgh jdraly2 Everyone needs to realise this.
 _________

Here, we can see a conversation developing! Now that we have this working, let's take a step back and display the shapes of our dataframes:

In [80]:
print(askReddit_df.shape)
print(askReddit_hot_df.shape)

(100, 5)
(100, 5)


In [81]:
#politics
print(politics_df.shape)
print(politics_hot_df.shape)


(25, 5)
(100, 5)


In [82]:
#soccer

print(soccer_df.shape)
print(soccer_hot_df.shape)



(23, 5)
(100, 5)


In [83]:
#uplifting news

print(uplifting_df.shape)
print(uplifting_hot_df.shape)


(16, 5)
(100, 5)


In [84]:
#music

print(music_df.shape)
print(music_hot_df.shape)

(23, 5)
(100, 5)


In less active subreddits, we see that there are fewer 'rising' posts